In [1]:
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
from shutil import copyfile
from sklearn.model_selection import KFold 
import cv2


In [2]:
image_labels_file = "nfl-health-and-safety-helmet-assignment/image_labels.csv"
image_labels = pd.read_csv(image_labels_file)
# image_labels['image']
# image_labels

In [7]:
def get_yolo_format_bbox(img_w, img_h, box):

    w = box.width.values  # width
    h = box.height.values  # height
    xc = box.left.values + np.round(w / 2)  # xmin + width/2
    yc = box.top.values + np.round(h / 2) # ymin + height/2
    # print(w)
    # print(xc)

    return pd.DataFrame({'x':xc / img_w, 'y':yc / img_h, 'w':w / img_w, 'h':h / img_h} ) # x_center y_center width height


def fold_csv2yolo(csv_root,yolo_root,train_label,val_label):
    TRAIN_IMAGES=f'{yolo_root}/images/train'
    VAL_IMAGES=f'{yolo_root}/images/val'
    TRAIN_LABELS=f'{yolo_root}/labels/train'
    VAL_LABELS=f'{yolo_root}/labels/val'


    if not os.path.exists(TRAIN_IMAGES): os.makedirs(TRAIN_IMAGES)
    if not os.path.exists(VAL_IMAGES): os.makedirs(VAL_IMAGES)
    if not os.path.exists(TRAIN_LABELS):os.makedirs(TRAIN_LABELS)
    if not os.path.exists(VAL_LABELS): os.makedirs(VAL_LABELS)

    train_image=train_label.image.drop_duplicates().values
    val_image=val_label.image.drop_duplicates().values

    for image in tqdm(train_image):
        copyfile(f'{csv_root}/images/{image}',f'{TRAIN_IMAGES}/{image}')

        label=train_label.loc[train_label['image']==image]
        box=label[['left','top','width','height']]
        # print(box)
        img=cv2.imread(f'{csv_root}/images/{image}')
        h,w,_=img.shape
        box=get_yolo_format_bbox(w,h,box)
        file=image.split('.')[0]
        label_file_name=f'{TRAIN_LABELS}/{file}.txt'
        with open(label_file_name,'w') as f:
            for i,data in box.iterrows():
                label=[str(i) for i in data.values]
                label=' '.join(label)
                label=str(0)+' '+label
                f.write(label)
                f.write('\n')

    for image in tqdm(val_image):
        copyfile(f'{csv_root}/images/{image}',f'{VAL_IMAGES}/{image}')

        label=val_label.loc[val_label['image']==image]
        box=label[['left','top','width','height']]
        # print(box)
        img=cv2.imread(f'{csv_root}/images/{image}')
        h,w,_=img.shape
        box=get_yolo_format_bbox(w,h,box)
        
        file=image.split('.')[0]
        label_file_name=f'{VAL_LABELS}/{file}.txt'
        with open(label_file_name,'w') as f:
            for i,data in box.iterrows():
                label=[str(i) for i in data.values]
                label=' '.join(label)
                label=str(0)+' '+label
                f.write(label)
                f.write('\n')

    # print(train_image,len(train_image))


n_splits=5
kf = KFold(n_splits=n_splits,shuffle=False) 
image=image_labels['image'].drop_duplicates().reset_index(drop=True).values
# print(image.shape)
fold_num=0
for train_image_idx,val_image_idx in kf.split(image):
    # print(train_image_idx)
    train_image,val_image=image[train_image_idx],image[val_image_idx]
    # print(train_image,len(train_image))
    train_image_label,val_image_label=image_labels.loc[image_labels.image.isin(train_image) ],image_labels.loc[image_labels.image.isin(val_image) ]
    # print(train_image_label)
    
    yolo_root=f'yolov5/datasets/fold{fold_num}'
    csv_root='nfl-health-and-safety-helmet-assignment'
    if not os.path.exists(yolo_root):os.makedirs(yolo_root)
    fold_csv2yolo(csv_root,yolo_root,train_image_label,val_image_label)
    fold_num+=1

    


100%|██████████| 1989/1989 [00:34<00:00, 57.12it/s]


In [1]:
%cd /home/shigb/nfl/yolov5
# map 0.95 r 0.8
!python train.py --img 1280 --batch 10  --project NFL_YoloV5 --epochs 10 --data nfl_fold1.yaml --name yolov5x6_fold1 --weights yolov5x6.pt

/home/shigb/nfl/yolov5
train: weights=yolov5x6.pt, cfg=, data=nfl_fold1.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=10, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=NFL_YoloV5, name=yolov5x6_fold1, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds
YOLOv5 🚀 v6.0-270-g0365379 torch 1.10.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv

In [2]:
!python train.py --img 1280 --batch 10  --project NFL_YoloV5 --epochs 10 --data nfl_fold2.yaml --name yolov5x6_fold2 --weights yolov5x6.pt

train: weights=yolov5x6.pt, cfg=, data=nfl_fold2.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=10, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=NFL_YoloV5, name=yolov5x6_fold2, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 6 commits. Use `git pull` or `git clone git@github.com:ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-270-g0365379 torch 1.10.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl

In [3]:
!python train.py --img 1280 --batch 10  --project NFL_YoloV5 --epochs 10 --data nfl_fold3.yaml --name yolov5x6_fold3 --weights yolov5x6.pt

train: weights=yolov5x6.pt, cfg=, data=nfl_fold3.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=10, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=NFL_YoloV5, name=yolov5x6_fold3, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 6 commits. Use `git pull` or `git clone git@github.com:ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-270-g0365379 torch 1.10.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl

In [4]:
!python train.py --img 1280 --batch 10  --project NFL_YoloV5 --epochs 10 --data nfl_fold4.yaml --name yolov5x6_fold4 --weights yolov5x6.pt

train: weights=yolov5x6.pt, cfg=, data=nfl_fold4.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=10, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=NFL_YoloV5, name=yolov5x6_fold4, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 6 commits. Use `git pull` or `git clone git@github.com:ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-270-g0365379 torch 1.10.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl